In [1]:
import random
import time

from bert import preprocess_function
from datasets import load_dataset
import numpy as np
from metrics import score_generated_sentences
from transformers import BertTokenizer, EncoderDecoderModel
import torch

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.7.crossattention.self.value.bias', 'bert.encoder.layer.8.crossattention.self.query.bias', 'bert.encoder.layer.9.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.11.crossattention.self.value.bias', 'bert.encoder.layer.10.crossattention.self.value.weight', 'bert.encoder.layer.11.crossattention.self.query.weight', 'bert.encoder.layer.11.crossattention.self.query.bias', 'bert.encoder.layer.4.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.8.crossattention.self.query.weight', 'bert.encoder.layer.11.crossattention.self.key.weight', 'bert.encoder.layer.10.crossattention.output.dense.bias', 'bert.encoder.layer.9.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.3.crossattention.self.value.weight', 'bert.encoder.layer.8.crossattention.output.dense.bias', 'bert.encoder.l

In [2]:
tokenizer = BertTokenizer.from_pretrained("./encoder")
model = EncoderDecoderModel.from_encoder_decoder_pretrained("./encoder", "./decoder")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.to(device)
device

device(type='cuda', index=0)

In [3]:
ds = load_dataset("csebuetnlp/BanglaParaphrase")
index = random.randint(0, 1000)
tokenized_test = ds["test"].select(range(500)).map(preprocess_function, batched=True)

def predict(examples):
    input_ids = torch.from_numpy(np.array(examples["input_ids"])).to(device)
    o = model.generate(input_ids=input_ids, max_new_tokens=128)
    os = []
    for i in o:
        os.append(" ".join(tokenizer.decode(i).split("[SEP]")[0].split(" ")[1:]))
    examples["output"] = os
    return examples

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
print("source", ds["test"][index:index + 1]["source"])
print("target", ds["test"][index:index + 1]["target"])
tokenized = tokenizer(ds["test"][index:index + 1]["source"], return_tensors="pt").to(device)
print("tokenized source:", tokenizer.decode(tokenized.input_ids[0]))
tokenized = tokenizer(ds["test"][index:index + 1]["target"], return_tensors="pt").to(device)
print("tokenized target:", tokenizer.decode(tokenized.input_ids[0]))

outputs = model.generate(input_ids=tokenized.input_ids, max_new_tokens=128)
print("output", tokenizer.decode(outputs[0]).split("[SEP]")[0])

source ['এতে থাকা অণুজীবগুলোকেই মূলত তিনি দায়ী করেন।']
target ['তিনি মূলত এর মধ্যে যে-অণুজীবগুলো রয়েছে, সেগুলোকে দোষ দেন।']
tokenized source: [CLS] এতে থাকা অণজীবগলোকেই মলত তিনি দাযী করেন । [SEP]
tokenized target: [CLS] তিনি মলত এর মধযে যে - অণজীবগলো রযেছে, সেগলোকে দোষ দেন । [SEP]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
started = time.time()
test = tokenized_test.map(lambda batch: predict(batch), batched=True, batch_size=256)
scores = score_generated_sentences(test["target"], test["output"])
print(scores)
print("scoring time (seconds):", time.time() - started)